In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/40"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 70 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

from transformers import AutoImageProcessor, ConvNextForImageClassification

processor = AutoImageProcessor.from_pretrained("facebook/convnext-small-224")
model = AutoModelForImageClassification.from_pretrained("facebook/convnext-small-224")

# Move model to device
model.to(device)

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Epoch 1/20, Train Loss: 3.1313, Train Accuracy: 32.45%, Val Loss: 1.8381, Val Accuracy: 45.91%


Epoch 2/20, Train Loss: 1.3945, Train Accuracy: 54.37%, Val Loss: 1.2329, Val Accuracy: 56.93%


Epoch 3/20, Train Loss: 0.9507, Train Accuracy: 67.91%, Val Loss: 0.8956, Val Accuracy: 68.61%


...

Epoch 16/20, Train Loss: 0.0580, Train Accuracy: 97.99%, Val Loss: 0.3051, Val Accuracy: 88.48%
Epoch 00016: reducing learning rate of group 0 to 1.0000e-05.


Epoch 17/20, Train Loss: 0.0551, Train Accuracy: 97.85%, Val Loss: 0.2538, Val Accuracy: 90.65%


Epoch 18/20, Train Loss: 0.0425, Train Accuracy: 98.14%, Val Loss: 0.2432, Val Accuracy: 91.32%


Epoch 19/20, Train Loss: 0.0393, Train Accuracy: 98.57%, Val Loss: 0.2432, Val Accuracy: 91.49%


Epoch 20/20, Train Loss: 0.0316, Train Accuracy: 98.71%, Val Loss: 0.2258, Val Accuracy: 91.99%
Test Accuracy: 91.99%


In [7]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 91.99%
Classification Report:
              precision    recall  f1-score   support

           0      0.783     0.973     0.867        37
           1      0.903     0.848     0.875        33
           2      0.977     0.824     0.894        51
           3      0.957     0.880     0.917        75
           4      0.829     0.872     0.850        39
           5      0.899     0.886     0.892        70
           6      0.935     0.965     0.950       255
           7      1.000     0.949     0.974        39

    accuracy                          0.920       599
   macro avg      0.910     0.899     0.902       599
weighted avg      0.923     0.920     0.920       599



In [8]:
# Train the model
num_epochs = 40
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/40, Train Loss: 0.0394, Train Accuracy: 98.21%, Val Loss: 0.2213, Val Accuracy: 91.82%


Epoch 22/40, Train Loss: 0.0395, Train Accuracy: 98.28%, Val Loss: 0.2148, Val Accuracy: 91.99%


Epoch 23/40, Train Loss: 0.0357, Train Accuracy: 98.57%, Val Loss: 0.2226, Val Accuracy: 91.32%


Epoch 24/40, Train Loss: 0.0331, Train Accuracy: 98.50%, Val Loss: 0.2202, Val Accuracy: 91.32%


Epoch 25/40, Train Loss: 0.0330, Train Accuracy: 98.57%, Val Loss: 0.2209, Val Accuracy: 91.82%


...

Epoch 36/40, Train Loss: 0.0339, Train Accuracy: 98.57%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 37/40, Train Loss: 0.0328, Train Accuracy: 98.57%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 38/40, Train Loss: 0.0344, Train Accuracy: 98.64%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 39/40, Train Loss: 0.0346, Train Accuracy: 98.50%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 40/40, Train Loss: 0.0327, Train Accuracy: 98.64%, Val Loss: 0.2212, Val Accuracy: 91.49%
Test Accuracy: 91.49%


In [9]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 91.49%
Classification Report:
              precision    recall  f1-score   support

           0      0.783     0.973     0.867        37
           1      0.900     0.818     0.857        33
           2      0.977     0.824     0.894        51
           3      0.932     0.907     0.919        75
           4      0.805     0.846     0.825        39
           5      0.897     0.871     0.884        70
           6      0.935     0.957     0.946       255
           7      1.000     0.949     0.974        39

    accuracy                          0.915       599
   macro avg      0.903     0.893     0.896       599
weighted avg      0.918     0.915     0.915       599



In [10]:
# Train the model
num_epochs = 60
for epoch in range(40, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 41/60, Train Loss: 0.0332, Train Accuracy: 98.35%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 42/60, Train Loss: 0.0353, Train Accuracy: 98.50%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 43/60, Train Loss: 0.0301, Train Accuracy: 98.78%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 44/60, Train Loss: 0.0326, Train Accuracy: 98.35%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 45/60, Train Loss: 0.0279, Train Accuracy: 98.78%, Val Loss: 0.2212, Val Accuracy: 91.49%


...

Epoch 56/60, Train Loss: 0.0318, Train Accuracy: 98.71%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 57/60, Train Loss: 0.0354, Train Accuracy: 98.42%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 58/60, Train Loss: 0.0290, Train Accuracy: 98.64%, Val Loss: 0.2212, Val Accuracy: 91.49%


Epoch 59/60, Train Loss: 0.0287, Train Accuracy: 98.78%, Val Loss: 0.2213, Val Accuracy: 91.49%


Epoch 60/60, Train Loss: 0.0316, Train Accuracy: 98.85%, Val Loss: 0.2213, Val Accuracy: 91.49%
Test Accuracy: 91.49%


In [11]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 91.49%
Classification Report:
              precision    recall  f1-score   support

           0      0.783     0.973     0.867        37
           1      0.900     0.818     0.857        33
           2      0.977     0.824     0.894        51
           3      0.932     0.907     0.919        75
           4      0.805     0.846     0.825        39
           5      0.897     0.871     0.884        70
           6      0.935     0.957     0.946       255
           7      1.000     0.949     0.974        39

    accuracy                          0.915       599
   macro avg      0.903     0.893     0.896       599
weighted avg      0.918     0.915     0.915       599

